#  Que peuvent faire les *transformers* ?

Installez la bibliothèque 🤗 *Transformers* pour exécuter ce *notebook*.

In [10]:
#!pip install transformers[sentencepiece]
#!pip install tensorflow

In [4]:
from transformers import pipeline

In [5]:
dir(pipeline)

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

### Analyse de sentiments

In [7]:
classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598048329353333}]

In [15]:
#classifier = pipeline("sentiment-analysis", model="tblard/tf-allocine")
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased")
classifier("J'ai attendu un cours d'HuggingFace toute ma vie.") 

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_1', 'score': 0.5260409116744995}]

Intéressant ! On observe que le résultat est négatif là où pour la version en anglais le résultat est positif.

In [16]:
classifier(
    ["J'ai attendu un cours d'HuggingFace toute ma vie.", 
     "Je déteste tellement ça !"]
) # pour classifier plusieurs phrases

[{'label': 'LABEL_1', 'score': 0.5260409116744995},
 {'label': 'LABEL_1', 'score': 0.5385863780975342}]

La phrase "J'ai attendu un cours d'HuggingFace toute ma vie." qui était précedemment négative devient à présent positive.

### Zéro shot classification

In [ ]:
classifier = pipeline("zero-shot-classification", model="BaptisteDoyen/camembert-base-xnli")
classifier(
    "C'est un cours sur la bibliothèque Transformers",
    candidate_labels=["éducation", "politique", "affaires"],
)

### Génération de texte

In [19]:
generator = pipeline("text-generation", model="asi/gpt-fr-cased-small")
generator("# Dans ce cours, nous vous enseignerons comment")

[{'generated_text': '# Dans ce cours, nous vous enseignerons comment nous pouvons être des êtres humains. " " Nous'}]

In [20]:
generator = pipeline("text-generation", model="asi/gpt-fr-cased-small")
generator(
    "# Dans ce cours, nous vous enseignerons comment",
    max_length=30,
    num_return_sequences=1,
)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': '# Dans ce cours, nous vous enseignerons comment nous pouvons être des êtres humains. " " Nous vous enseignerons comment nous pouvons être des êtres humains'}]

### Remplacement des mots masqués

In [22]:
unmasker = pipeline("fill-mask", model="camembert-base")
unmasker(" Ce cours vous apprendra tout sur les modèles <mask>.", top_k=2)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.10731782764196396,
  'token': 15328,
  'token_str': 'automobiles',
  'sequence': 'Ce cours vous apprendra tout sur les modèles automobiles.'},
 {'score': 0.07520482689142227,
  'token': 4007,
  'token_str': 'électriques',
  'sequence': 'Ce cours vous apprendra tout sur les modèles électriques.'}]

### Reconnaissance d'entités nommées

In [24]:
ner = pipeline("ner", model="Jean-Baptiste/camembert-ner", grouped_entities=True)
ner("Je m'appelle Sylvain et je travaille à Hugging Face à Brooklyn.")

[{'entity_group': 'PER',
  'score': 0.60744476,
  'word': 'Sylvain',
  'start': 12,
  'end': 20},
 {'entity_group': 'LOC',
  'score': 0.6868405,
  'word': 'Hugging Face',
  'start': 38,
  'end': 51},
 {'entity_group': 'LOC',
  'score': 0.9952378,
  'word': 'Brooklyn',
  'start': 53,
  'end': 62}]

### Réponse à des questions

In [26]:
question_answerer = pipeline("question-answering", model="etalab-ia/camembert-base-squadFR-fquad-piaf")
question_answerer(
    question="Où est-ce que je travaille ?",
    context="Je m'appelle Sylvain et je travaille à Hugging Face à Brooklyn.",
)

{'score': 0.5418344736099243,
 'start': 38,
 'end': 63,
 'answer': ' Hugging Face à Brooklyn.'}

###  Résumé

In [28]:
summarizer = pipeline("summarization", model="moussaKam/barthez-orangesum-abstract")
summarizer(
    """
    L'Amérique a changé de façon spectaculaire au cours des dernières années. Non seulement le nombre de 
    diplômés dans les disciplines traditionnelles de l'ingénierie telles que le génie mécanique, civil, 
    l'électricité, la chimie et l'aéronautique a diminué, mais dans la plupart 
    des grandes universités américaines, les programmes d'études d'ingénierie se concentrent désormais sur 
    et encouragent largement l'étude des sciences de l'ingénieur. Par conséquent, il y a 
    de moins en moins d'offres dans les sujets d'ingénierie traitant de l'infrastructure, 
    l'environnement et les questions connexes, et une plus grande concentration sur les sujets de haute 
    technologie, qui soutiennent en grande partie des développements scientifiques de plus en plus 
    complexes. Si cette dernière est importante, elle ne doit pas se faire au détriment
    de l'ingénierie plus traditionnelle.

    Les économies en développement rapide telles que la Chine et l'Inde, ainsi que d'autres 
    pays industrialisés d'Europe et d'Asie, continuent d'encourager et de promouvoir
    l'enseignement de l'ingénierie. La Chine et l'Inde, respectivement, diplôment 
    six et huit fois plus d'ingénieurs traditionnels que les États-Unis. 
    Les autres pays industriels maintiennent au minimum leur production, tandis que l'Amérique 
    souffre d'une baisse de plus en plus importante du nombre de diplômés en ingénierie
    et un manque d'ingénieurs bien formés.
"""
)

[{'summary_text': "Les universités américaines dispensent désormais des cours d'ingénierie centrés sur les sujets de l'infrastructure, l'environnement et les questions connexes."}]

###  Traduction

In [30]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")
translator("This course is produced by Hugging Face.")

[{'translation_text': 'Ce cours est produit par Hugging Face.'}]